In [ ]:
# shape files
import requests
import os
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import json
# DRIVE https://drive.google.com/drive/folders/1KsIfAFmp0ArLauvHY1k9wRc9ZXaPDahe
# Make a GET request to the API endpoint
# Example usage: Waterworks District Boundries
api_url = 'https://dpw.gis.lacounty.gov/dpw/rest/services/PW_Open_Data/MapServer/21/query?where=1%3D1&outFields=*&outSR=4326&f=json'
district = "sample"
try:
    response = requests.get(api_url)
    response.raise_for_status()  # Will raise an HTTPError for bad requests (4XX, 5XX)
except requests.RequestException as e:
    print(f"Request failed: {e}")

# Parse JSON response
try:
    data = response.json()
except json.JSONDecodeError:
    print("Failed to decode JSON from response")

if 'features' in data:
    features = data['features']
    properties_list = []
    geometry_list = []

    # Process each feature to extract geometry and attributes
    for feature in features:
        attributes = feature['attributes']
        properties_list.append(attributes)
        
        # Construct the geometry using shapely
        if 'geometry' in feature and 'rings' in feature['geometry']:
            polygon = Polygon([tuple(l) for l in feature['geometry']['rings'][0]])
            geometry_list.append(polygon)
        else:
            geometry_list.append(None)

In [ ]:
import numpy as np
from shapely.geometry import Polygon
from sklearn.decomposition import TruncatedSVD

def compress_geometries(geometry_list, n_components=5):
    # Flatten geometry to a numpy array
    coord_list = [np.array(poly.exterior.coords).ravel() for poly in geometry_list if poly is not None]
    max_len = max(len(coords) for coords in coord_list)
    data_matrix = np.array([np.pad(coords, (0, max_len - len(coords)), 'constant') for coords in coord_list])
    
    # Apply SVD
    svd = TruncatedSVD(n_components=n_components)
    reduced_data = svd.fit_transform(data_matrix)
    
    return reduced_data, svd

# Example use with your geometry list
compressed_data, svd_model = compress_geometries(geometry_list)